# Rijksmuseum API

In [84]:
import requests
import re

def search_portraits(title=None, creator=None, t='painting'):
    SEARCH_URL = "https://data.rijksmuseum.nl/search/collection"

    params = {
        "creator": creator, 
        "title": title,
        "imageAvailable": "true",
        "type": t
    }

    r = requests.get(SEARCH_URL, params=params)
    r.raise_for_status()
    data = r.json()

    return data

In [85]:
def parse_artwork_details(data: dict) -> dict:
    """
    Extracts useful structured fields from Rijksmuseum Linked.Art objects
    """
    
    en_code = "http://vocab.getty.edu/aat/300388277" # prefer English so there is no need to find a way for translation
    nl_code = "http://vocab.getty.edu/aat/300388256" # dutch language has more information

    unit_map = {
        "http://vocab.getty.edu/aat/300379098": "cm",
        "http://vocab.getty.edu/aat/300379226": "kg",
    }
    
    attr_map = {
        "https://id.rijksmuseum.nl/22011": "hoogte",
        "https://id.rijksmuseum.nl/22012": "breedte",
        "https://id.rijksmuseum.nl/220217": "gewicht",
    }

    # ------------ TITLE ------------
    # First look for Dutch version
    
    title = None
    for s in data.get("subject_of", []):
        for part in s.get("part", []):
            for sub in part.get("part", []):
                if sub.get("type") == "Name":
                    langs = sub.get("language", [])
                    if any(l.get("id") == en_code for l in langs):
                        title = sub.get("content")
                        break
            if title:
                break
        if title:
            break

    # fallback: take any title if no English was found
    if not title:
        for s in data.get("subject_of", []):
            for part in s.get("part", []):
                for sub in part.get("part", []):
                    if sub.get("type") == "Name":
                        title = sub.get("content")
                        break
                if title:
                    break
            if title:
                break

    # ------------ ARTIST / MAKER ------------
    artist_name = None
    artist_id = None
    
    prod = data.get("produced_by")
    if isinstance(prod, dict):
        for part in prod.get("part", []):
            # get the person URI
            for agent in part.get("carried_out_by", []):
                artist_id = agent.get("id")
    
            # read Dutch referred_to_by labels
            for ref in part.get("referred_to_by", []):
                if ref.get("type") == "LinguisticObject":
                    langs = ref.get("language", [])
                    if any(l.get("id") == en_code for l in langs):
                        artist_name = ref.get("content")
                        break
    
            # fallback: any referred_to_by without language filter
            if artist_name is None:
                for ref in part.get("referred_to_by", []):
                    if ref.get("type") == "LinguisticObject":
                        artist_name = ref.get("content")
                        break

    # ------------ YEAR ------------
    year = None
    ts = prod.get("timespan") if prod else None
    if isinstance(ts, dict):
        # Try identified_by textual year first
        if isinstance(ts.get("identified_by"), list):
            for ident in ts["identified_by"]:
                c = ident.get("content")
                if c and any(ch.isdigit() for ch in c):
                    year = c
                    break

        # fallback to machine timestamps
        if year is None:
            b = ts.get("begin_of_the_begin")
            if b: 
                year = b[:4]

    # ------------ DESCRIPTION ------------
    descriptions_nl = []

    for entry in data.get("subject_of", []):
        langs = entry.get("language", [])
        if not any(l.get("id") == en_code for l in langs):
            continue
    
        # level 1: direct content
        if "content" in entry:
            descriptions_nl.append(entry["content"])
    
        # level 2: parts
        for p in entry.get("part", []):
            if "content" in p:
                descriptions_nl.append(p["content"])
            for sub in p.get("part", []):
                if "content" in sub:
                    descriptions_nl.append(sub["content"])
    # deduplicate
    descriptions_nl = list(dict.fromkeys(descriptions_nl))

    description = " ".join(descriptions_nl)

    # ------------ LOCATION ------------
    location = None
    room = None
    loc = data.get('current_location', [])

    if loc:
        for item in loc.get("identified_by", []):
        
            # 1. Extract identifier
            if item.get("type") == "Identifier":
                if "content" in item:
                    room = item["content"]
        
            # 2. Extract location name in english
            if item.get("type") == "Name":
                langs = item.get("language", [])
                if any(l.get("id") == en_code for l in langs):
                    parts = item.get("part", [])
                    names = [p.get("content") for p in parts if p.get("content")]
                    location = " ".join(names)
                    
    # ------------ DIMENSION ------------        
    entries = []
    
    for item in data.get("dimension", []):
        if item.get("type") != "Dimension":
            continue
        
        value = item.get("value")
        unit_id = item.get("unit", {}).get("id")
        unit = unit_map.get(unit_id, "")
        
        # get attribute from classified_as
        attr = None
        for c in item.get("classified_as", []):
            a = attr_map.get(c.get("id"))
            if a:
                attr = a
        
        # gather the Dutch annotation text
        annotation = None
        for ref in item.get("referred_to_by", []):
            langs = ref.get("language", [])
            if any(l.get("id") == en_code for l in langs):
                annotation = ref.get("content")
    
        if attr and value and unit:
            entries.append(f"{attr} {value} {unit}" + (f" ({annotation})" if annotation else ""))
            
    dimension_str = " x ".join(entries)

    # ------------ MATERIAL ------------        

    material_code = "http://vocab.getty.edu/aat/300435429"
    
    materials = []
    
    for item in data.get("referred_to_by", []):
        if item.get("type") != "LinguisticObject":
            continue
        
        langs = item.get("language", [])
        if not any(l.get("id") == en_code for l in langs):
            continue
    
        classes = item.get("classified_as", [])
        if not any(c.get("id") == material_code for c in classes):
            continue
    
        content = item.get("content")
        if content:
            materials.append(content)
    
    materials = list(dict.fromkeys(materials))

    
    return {
        "title": title,
        "artist": artist_name,
        "year": year,
        "description": description,
        "location": location,
        "room": room,
        "dimension": dimension_str,
        "material": materials,
        "source": data.get("id"),
    }

### Retrieve metadata for selected artwork

In [142]:
# what to search - example
creator = 'Vermeer'
title = 'Milkmaid'

In [143]:
data = search_portraits(title=title, creator=creator)
rijks_artwork_id = data["orderedItems"][0]['id']

extracted_info = requests.get(rijks_artwork_id, headers={"Accept": "application/ld+json"}).json()

extracted_data = parse_artwork_details(extracted_info)

In [144]:
extracted_data  # what we extract

{'title': 'The Milkmaid',
 'artist': 'painter: Johannes Vermeer',
 'year': 'c. 1660',
 'description': 'A maidservant pours milk, entirely absorbed in her work. Except for the stream of milk, everything else is still. Vermeer took this simple everyday activity and made it the subject of an impressive painting – the woman stands like a statue in the brightly lit room. Vermeer also had an eye for how light by means of hundreds of colourful dots plays over the surface of objects. Johannes Vermeer (1632–1675), oil on canvas, c. 1660 The Milkmaid',
 'location': 'Main building Gallery of Honour',
 'room': 'HG-2.30.3',
 'dimension': 'breedte 41 cm x hoogte 45.5 cm',
 'material': ['oil on canvas'],
 'source': 'https://id.rijksmuseum.nl/200108369'}

### Retrieve metadata of other artworks of the same artist

In [152]:
rel_artworks = []
data_artist = search_portraits(creator=creator)
if len(data_artist['orderedItems']) > 1:
    for items in data_artist['orderedItems']:
        if (rijks_artwork_id != items['id']):
            rel_art_id = items['id']
            rel_art_extracted_info = requests.get(rel_art_id, headers={"Accept": "application/ld+json"}).json()
            rel_art_extracted_data = parse_artwork_details(rel_art_extracted_info)
            rel_artworks.append(rel_art_extracted_data)

In [153]:
print(f'Other artworks of {creator}: \n')
for d in rel_artworks:
    print(d['title'])

Other artworks of Vermeer: 

None
View of Houses in Delft, Known as ‘The Little Street’
Woman Reading a Letter
The Love Letter


# Retrieve data from Wikipedia

In [93]:
def wikidata_search(title):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbsearchentities",
        "language": "en",
        "format": "json",
        "search": title
    }
    headers = {"User-Agent": "RijksmuseumRAGBot/1.0 (https://example.com; contact@example.com)"}
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()
    return r.json()["search"]

In [94]:
def wikidata_get(qid):
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    headers = {"User-Agent": "RijksmuseumRAGBot/1.0 (https://example.com; contact@example.com)"}
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    return r.json()["entities"][qid]

In [95]:
def select_painting(results):
    for item in results:
        qid = item["id"]
        entity = wikidata_get(qid)
        claims = entity.get("claims", {})
        if "P31" in claims:
            for inst in claims["P31"]:
                if inst["mainsnak"]["datavalue"]["value"]["id"] == "Q3305213":
                    return qid
    return None

In [96]:
results = wikidata_search(title)
qid = select_painting(results)
print(qid)

Q167605


In [97]:
def wikidata_get_sitelink(qid, lang="en"):
    url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    headers = {
        "User-Agent": "RijksmuseumRAGBot/1.0 (https://example.com; contact@example.com)"
    }
    r = requests.get(url, headers=headers)
    r.raise_for_status()
    data = r.json()
    entity = data["entities"][qid]
    return entity["sitelinks"][f"{lang}wiki"]["title"]

In [98]:
def wikipedia_content(title, lang="en"):
    url = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,    # remove HTML
        "format": "json",
        "titles": title
    }
    headers = {
        "User-Agent": "RijksmuseumRAGBot/1.0 (https://example.com; contact@example.com)"
    }
    r = requests.get(url, params=params, headers=headers)
    r.raise_for_status()
    data = r.json()
    pages = data["query"]["pages"]
    page = next(iter(pages.values()))
    return page.get("extract", "")

In [99]:
wiki_title = wikidata_get_sitelink(qid)
print(wiki_title)

The Milkmaid (Vermeer)


In [100]:
wiki_artwork_content = wikipedia_content(wiki_title, lang="en")
print(wiki_artwork_content[:300])

The Milkmaid (Dutch: De melkmeid or Het melkmeisje), sometimes called The Kitchen Maid (Dutch: De keukenmeid), is an oil-on-canvas painting of a "milkmaid", in fact, a domestic kitchen maid, by the Dutch artist Johannes Vermeer. It is in the Rijksmuseum in Amsterdam, the Netherlands, which regards i


In [101]:
def select_artist(results):
    for item in results:
        qid = item["id"]
        entity = wikidata_get(qid)
        claims = entity.get("claims", {})

        # check instance of = human (Q5)
        if "P31" in claims:
            if any(inst["mainsnak"]["datavalue"]["value"]["id"] == "Q5"
                   for inst in claims["P31"]):
                return qid
    return None

In [102]:
artist_results = wikidata_search(creator)
artist_qid = select_artist(artist_results)
artist_wiki_title = wikidata_get_sitelink(artist_qid, lang="en")
wiki_artist_bio = wikipedia_content(artist_wiki_title)

print(artist_wiki_title)
print(wiki_artist_bio[:300])

Johannes Vermeer
Johannes Vermeer ( vər-MEER, vər-MAIR, Dutch: [joːˈɦɑnəs fərˈmeːr]; see below; also known as Jan Vermeer; October 1632 – 15 December 1675) was a Dutch painter who specialized in domestic interior scenes of middle-class life. He is considered one of the greatest painters of the Dutch Golden Age. Duri


### Aggregate info

In [103]:
extracted_data

{'title': 'The Milkmaid',
 'artist': 'painter: Johannes Vermeer',
 'year': 'c. 1660',
 'description': 'A maidservant pours milk, entirely absorbed in her work. Except for the stream of milk, everything else is still. Vermeer took this simple everyday activity and made it the subject of an impressive painting – the woman stands like a statue in the brightly lit room. Vermeer also had an eye for how light by means of hundreds of colourful dots plays over the surface of objects. Johannes Vermeer (1632–1675), oil on canvas, c. 1660 The Milkmaid',
 'location': 'Main building Gallery of Honour',
 'room': 'HG-2.30.3',
 'dimension': 'breedte 41 cm x hoogte 45.5 cm',
 'material': ['oil on canvas'],
 'source': 'https://id.rijksmuseum.nl/200108369'}

In [104]:
def aggregate_data(df, wiki_artwork_content, wiki_artist_bio, rel_artworks):
    final_data = df.copy()
    final_data['wiki_artwork'] = wiki_artwork_content
    final_data['wiki_artist'] = wiki_artist_bio
    final_data['artist_artworks'] = rel_artworks
    return final_data

### Final Dataset (Merging all above)

In [106]:
search_set = {'Johannes Vermeer': ['The Milkmaid', 'The Love Letter', 'The Little Street'], 'Van Gogh': ['Self-Portrait']}

In [181]:
def data_extraction(search_set):
    artworks_data = {}
    for creator, titles in search_set.items():
        for title in titles:
            print(f'Scraping info for artwork "{title}" of {creator}')
            
            data = search_portraits(title=title, creator=creator)
            rijks_artwork_id = data["orderedItems"][0]['id']
            actual_id = re.search(r'/(\d+)(?:\?|$)', rijks_artwork_id).group(1)
    
            extracted_info = requests.get(rijks_artwork_id, headers={"Accept": "application/ld+json"}).json()
            
            extracted_data = parse_artwork_details(extracted_info)
            extracted_data['artist'] = extracted_data['artist'].replace("painter: ", "").strip() # cleaning

            # find all the other artworks from the artist
            rel_artworks = []
            data_artist = search_portraits(creator=creator)
            if len(data_artist['orderedItems']) > 0:
                for items in data_artist['orderedItems']:
                    if rijks_artwork_id != items['id']:
                        rel_art_id = items['id']
                        rel_art_extracted_info = requests.get(rel_art_id, headers={"Accept": "application/ld+json"}).json()
                        rel_art_extracted_data = parse_artwork_details(rel_art_extracted_info)
                        rel_art_extracted_data['artist'] = rel_art_extracted_data['artist'].replace("painter: ", "").strip() # cleaning
                        rel_art_extracted_data = {k: rel_art_extracted_data[k] for k in ['title', 'room', 'location', 'artist']}
                        if rel_art_extracted_data['title'] is not None:
                            rel_artworks.append(rel_art_extracted_data)
            if (title != 'Self-Portrait') and (creator != 'Van Gogh'): # edge case cause self portrait has multiple paintings not a specific one
                results = wikidata_search(title)
                qid = select_painting(results)
                wiki_title = wikidata_get_sitelink(qid)
            else:
                wiki_title = 'https://en.wikipedia.org/wiki/Portraits_of_Vincent_van_Gogh' # retrieve info of the whole category
            wiki_artwork_content = wikipedia_content(wiki_title, lang="en")

            # wiki for artist
            artist_results = wikidata_search(creator)
            artist_qid = select_artist(artist_results)
            artist_wiki_title = wikidata_get_sitelink(artist_qid, lang="en")
            wiki_artist_bio = wikipedia_content(artist_wiki_title)
            painting_data = aggregate_data(extracted_data, wiki_artwork_content, wiki_artist_bio, rel_artworks)
            
            artworks_data[actual_id] = painting_data
    return artworks_data

In [182]:
all_data = data_extraction(search_set)

Scraping info for artwork "The Milkmaid" of Johannes Vermeer
Scraping info for artwork "The Love Letter" of Johannes Vermeer
Scraping info for artwork "The Little Street" of Johannes Vermeer
Scraping info for artwork "Self-Portrait" of Van Gogh


In [128]:
all_data.keys()

dict_keys(['200108369', '200108370', '200108371', '200109794'])

In [188]:
# save data extraction json
import json
import os

directory_path = "Data"
os.makedirs(directory_path, exist_ok=True)

save_path = "Data/extracted_data.json"

with open(save_path, "w", encoding="utf-8") as f:
    json.dump(all_data, f, ensure_ascii=False, indent=2)

# RAG

In [ ]:
import textwrap
import json
import chromadb
from dotenv import load_dotenv
from openai import OpenAI
import os

In [2]:
# load json data
load_path = "Data/extracted_data.json"
all_data = json.load(open(load_path, encoding="utf-8"))

In [3]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [266]:
chroma = chromadb.PersistentClient(path="./db_rijksmuseum")
collection = chroma.get_or_create_collection("rijksmuseum_data")

In [267]:
def chunk_text(text, size=800):    
    text = text.replace("\n", " ")
    return textwrap.wrap(text, size)

In [268]:
def prepare_chunks(painting):
    chunks = []

    meta = f"""
    Title: {painting['title']}
    Artist: {painting['artist']}
    Year: {painting['year']}
    Room: {painting['room']}
    Location: {painting['location']}
    Material: {painting['material']}
    Dimensions: {painting['dimension']}
    """
    chunks.append({"type": "metadata", "text": meta})

    chunks.extend({"type": "curatorial", "text": c} 
                  for c in chunk_text(painting["description"]))

    chunks.extend({"type": "wiki_painting", "text": c}
                  for c in chunk_text(painting["wiki_artwork"]))

    return chunks

In [269]:
def index_artist_bio(painting):
    # index the artist info from wiki
    
    artist = painting['artist']
    bio_chunks = chunk_text(painting['wiki_artist'])

    for i, chunk in enumerate(bio_chunks):
        chunk_id = f"artist_{artist}_{i}"

        collection.upsert(
            ids=[chunk_id],
            embeddings=[embed(chunk)],
            metadatas=[{
                "artist": artist,
                "type": "wiki_artist_bio"
            }],
            documents=[chunk]
        )

In [270]:
def index_artist_artworks(painting, painting_id):
    # index the other artworks of the artist in the Rijksmuseum
    
    artist = painting["artist"]

    for i, art in enumerate(painting.get("artist_artworks", [])):
        text = f"""
        Other artworks by the creator in the Rijksmuseum:
        Title: {art['title']}
        Artist: {art['artist']}
        Location: {art['location']}
        Room: {art['room']}
        """
        chunk_id = f"{painting_id}_artist_artwork_{i}"

        collection.upsert(
            ids=[chunk_id],
            embeddings=[embed(text)],
            documents=[text],
            metadatas=[{
                "type": "artist_other_artwork",
                "artist": artist,
                "source_painting_id": painting_id,
                "artwork_title": art["title"]
            }]
        )


In [271]:
def embed(text):
    resp = client.embeddings.create(
        model="text-embedding-3-large",
        input=text)
    return resp.data[0].embedding

In [272]:
def index_painting(painting, painting_id):
    """index the whole painting with the 3 parts of information: rijksmuseum data + wiki info of artwork,
    wiki info of the artist, relevant artworks of the artist"""
    
    index_artist_bio(painting)
    chunks = prepare_chunks(painting)
    for i, chunk in enumerate(chunks):
        chunk_id = f"{painting_id}_{i}"
        collection.upsert(
            ids=[chunk_id],
            embeddings=[embed(chunk["text"])],
            documents=[chunk["text"]],
            metadatas=[{
                "painting_id": painting_id,
                "title": painting["title"],
                "artist": painting["artist"],
                "type": chunk["type"]
            }]
        )
    index_artist_artworks(painting, painting_id)

In [273]:
indexed_ids = set(collection.get()['ids'])
for painting_id, painting in all_data.items():
    # check if any chunk for this painting exists using prefix match

    prefix = painting_id + "_"
    if any(cid.startswith(prefix) for cid in indexed_ids):
    # if any(painting_id in cid for cid in indexed_ids):
        print(f"✔ Already indexed: {painting_id}")
        continue

    print(f"Indexing: {painting_id}")
    index_painting(painting, painting_id)

✔ Already indexed: 200108369
✔ Already indexed: 200108370
✔ Already indexed: 200108371
✔ Already indexed: 200109794


In [274]:
# ensure that we retrieve documents only for the specific artwork, or the artist, or descriptive info of his relevant artworks
def retrieve(query, creator, painting_id, k=8):
    query_emb = embed(query)

    return collection.query(
        query_embeddings=[query_emb],
        n_results=k,
        where={
            "$or": [
                {"painting_id": painting_id},
                {
                    "$and": [
                        {"type": "artist_other_artwork"},
                        {"source_painting_id": painting_id}
                    ]
                },
                {
                    "$and": [
                        {"type": "wiki_artist_bio"},
                        {"artist": creator}
                    ]
                }
            ]
        }
    )

In [275]:
def answer(query, title, creator, painting_id):
    results = retrieve(query, creator, painting_id, k=10)
    context = "\n\n".join(results["documents"][0])

    prompt = f"""
    You are an expert Rijksmuseum art assistant. Suppose that when the user asks you a question, he is already in the Rijksmuseum. You can answer questions ONLY about the artwork: {title} and the creator {creator}.
    
    User question:
    {query}
    
    Context:
    {context}
    
    Answer using ONLY the context above. If not answerable, say "I don't know from available information."
    If it is irrelevant to the artwork and the creator, you will politely respond that your purpose is to provide information only about the painting and the artist.

    """

    completion = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content

In [277]:
all_data.keys()

dict_keys(['200108369', '200108370', '200108371', '200109794'])

In [278]:
painting_id = '200108369'
title = all_data[painting_id]['title']
creator = all_data[painting_id]['artist']

print(title, creator, painting_id)

The Milkmaid Johannes Vermeer 200108369


In [279]:
print(answer(f"Which of his artworks we are not certain that belong to him but are actually attributed to him?", title, creator, painting_id))

Johannes Vermeer is known for having produced fewer than 50 paintings, with only 34 universally attributed to him today. In the 19th century, following his rediscovery, more than 70 works were ascribed to Vermeer by Théophile Thoré-Bürger, who also regarded many of these attributions as uncertain. Over time, the number of works believed to be authentically by Vermeer has decreased as art historians have re-examined the evidence and clarified which paintings can reliably be said to be his. Therefore, there are several paintings historically attributed to Vermeer whose authenticity remains uncertain or debated, and not all works once thought to be by him are still universally accepted as such.

If you would like information on "The Milkmaid," which is universally accepted as Vermeer's work, I am happy to help!


In [280]:
print(answer("Where was Vermeer born and on which year?", title, creator, painting_id))

Johannes Vermeer was born in the city of Delft and was baptized on 31 October 1632.


In [20]:
print(answer("Describe Vermeer's artistic style.", title, creator, painting_id))

Johannes Vermeer’s artistic style is renowned for its extraordinary precision, mastery of light, and careful composition. He specialized in domestic interior scenes of middle-class life, often depicting one or two figures, usually women, lit by a window on the left. Vermeer's paintings are characterized by a sense of compositional balance and spatial order, unified by a sense of “pearly” natural light.

Vermeer may have first executed his paintings tonally, using monochrome shades of grey (“grisaille”) or limited palettes, over which he applied more saturated colors—particularly reds, yellows, and blues—in transparent glazes. He is especially noted for his lavish use of the expensive pigment ultramarine, derived from lapis lazuli, and for creating luminous effects with lead-tin-yellow.

His works, including “Het melkmeisje,” offer photographic-like realism and a sense of “tactile illusionism,” imbuing ordinary domestic activities with poetic timelessness. Vermeer’s attention to surface

In [21]:
print(answer("Why did he choose to paint it that way?", title, creator, painting_id))

Johannes Vermeer chose to paint Het melkmeisje in such a way to achieve a sense of monumentality and dignity in the depiction of a simple milkmaid engaged in honest work. He used a relatively low vantage point and built up the forms in a pyramidal composition, focusing the viewer’s attention on the woman’s right wrist, according to the Rijksmuseum and The Metropolitan Museum of Art.

Vermeer was also intentional with his use of color and light. He employed an exceptionally luminous color scheme, making extensive use of very expensive pigments like natural ultramarine (made from crushed lapis lazuli) and lead-tin-yellow, which set his work apart from his contemporaries. This choice created vibrant blues and yellows that contrasted with the more typical, muted palettes of other painters of the time. Vermeer’s meticulous technique, layering colors and using glazes, helped him depict different textures—from the coarse fabric of the maid’s clothing to the smoothness of the bread and walls.


In [22]:
print(answer("Which football club is the best in the Netherlands?", title, creator, painting_id))

My purpose is to provide information only about the artwork Het melkmeisje and the creator Johannes Vermeer. If you have any questions about the painting or the artist, I would be delighted to help!


In [23]:
print(answer(f"Which are other paintings by {creator} in the Rijksmuseum? show me the rooms of each", title, creator, painting_id))

Johannes Vermeer has several paintings in the Rijksmuseum, in addition to Het melkmeisje. They are all located in the Eregalerij of the Hoofdgebouw, specifically in room HG-2.30.3. Here is the list:

1. Het melkmeisje (The Milkmaid) – Room HG-2.30.3
2. Gezicht op huizen in Delft, bekend als ‘Het straatje’ (View of Houses in Delft, known as 'The Little Street') – Room HG-2.30.3
3. Brieflezende vrouw (Woman Reading a Letter) – Room HG-2.30.3
4. De liefdesbrief (The Love Letter) – Room HG-2.30.3

All of these Vermeer works in the Rijksmuseum can be found together in Room HG-2.30.3 of the Eregalerij, Hoofdgebouw.


In [26]:
painting_id = '200109794'
title = all_data[painting_id]['title']
creator = all_data[painting_id]['artist']

print(title, creator, painting_id)

Zelfportret Vincent van Gogh 200109794


In [27]:
print(answer(f"Which are other paintings by {creator} in the Rijksmuseum? show me the rooms of each", title, creator, painting_id))

The other paintings by Vincent van Gogh in the Rijksmuseum, besides Zelfportret, are:

1. Korenveld – Room: HG-1.18
2. Het Singel bij de Lutherse Kerk te Amsterdam – Room: HG-1.18
3. Oever met bomen – Room: HG-1.18

You can find all of these works, including Zelfportret, in room HG-1.18 of the Rijksmuseum.


In [19]:
print(answer("What where his thoughts when painting this portrait?", title, creator, painting_id))

When Vincent van Gogh painted his self-portraits, such as "Zelfportret," his thoughts were often introspective. He created self-portraits during times when he was reluctant to mix with others or lacked other models. For Van Gogh, painting himself was a way to study his own character and emotion, and these works reflected a high degree of self-scrutiny. He once wrote that portraiture was his greatest passion, saying, "What I'm most passionate about, much much more than all the rest in my profession, is the portrait, the modern portrait."

Van Gogh intended for his portraits, including his self-portraits, to go beyond likeness and instead capture emotion and inner character through his use of colour and brushwork. He described paintings he was satisfied with as "purposeful" canvases, using colour and technique to express something deeper. During periods of emotional difficulty or isolation, painting self-portraits served as a way for him to expose himself to new visual challenges and to 

### Predefined questions

In [ ]:
import numpy as np
from numpy.linalg import norm

In [72]:
predefined_questions = {
    '200108369': [
        "Why are you likely making bread pudding?",
        "How do you dignify the maid figure?",
        "What symbolic meanings surround the foot warmer?",
        "How do diagonals direct viewer attention?",
        "How is domestic virtue expressed here?",
        "Why is ultramarine significant in this painting?",
        "How do you create sculptural realism with light?",
        "Why did you remove background objects?",
        "What erotic symbols are subtly present?",
        "How is ambiguity conveyed in the maid’s expression?",
        "How do maid tropes influence interpretation?",
        "What social anxieties surround maids here?",
        "What identifies the room as a cold kitchen?",
        "How does texture enhance illusionism?",
        "What challenges the camera obscura theory?",
        "How does Pepys contextualize maid desire?",
        "Why is 'melkmeisje' preferred over 'keukenmeid'?",
        "How does vantage point convey dignity?",
        "Why include stale bread on the table?",
        "How do critics describe its museum significance?",
        "What marks this as a transitional work?",
        "How does tactility affect viewer perception?",
        "Which artists influenced maid iconography?",
        "How do desire and virtue coexist here?",
        "What was the painting's provenance journey?",
        "How do glazes differentiate clothing textures?",
        "How do color choices shape atmosphere?",
        "What values are implied by careful cooking?",
        "Why is the emotional state unreadable?",
        "How does the subject reflect Dutch social changes?"
    ],

    '200108370': [
        "Why is the viewpoint deliberately veiled?",
        "What does the curtain imply about privacy?",
        "How do you use the maid to mediate narrative?",
        "What emotions does the mistress display?",
        "How is class difference visually encoded?",
        "Why is the cittern symbolically significant?",
        "How does the seascape function metaphorically?",
        "Why include a landscape above the seascape?",
        "What do the slippers imply erotically?",
        "How does the brush signal neglected domestic duty?",
        "How do blue and gold structure the palette?",
        "What indicates household wealth?",
        "How do you create spatial depth?",
        "Why is this your only seascape?",
        "What cultural anxieties surround love letters?",
        "Why might the lover be considered absent?",
        "What role does anticipation play in the scene?",
        "How do diagonals guide the gaze?",
        "What is implied by the maid’s expression?",
        "How does costume communicate social status?",
        "What narrative is suggested by nautical allegory?",
        "How was the painting stolen in 1971?",
        "Why did the thief demand famine relief?",
        "What damage occurred during the theft?",
        "How long did restoration require afterward?",
        "Where did the painting once reside in Poland?",
        "What does provenance reveal about its prestige?",
        "How do props encode gender expectations?",
        "How do you balance virtue and desire?"
    ],

    '200108371': [
        "Why depict ordinary Delft houses?",
        "How is domestic labor implied?",
        "What makes the composition unusually balanced?",
        "How does texture convey material authenticity?",
        "Why emphasize brickwork so palpably?",
        "How do straight angles create visual dynamism?",
        "What do the children suggest about daily life?",
        "How is quietness communicated?",
        "What distinguishes this from genre interiors?",
        "How does weather affect mood?",
        "Why was the location long debated?",
        "How did research settle the location in 2015?",
        "Why was Vlamingstraat identified?",
        "How is the Penspoort referenced?",
        "What role did your aunt play here?",
        "How does family property shape subject choice?",
        "Why paint a humble street scene?",
        "What does this reveal about Delft urban fabric?",
        "How does scale influence viewer intimacy?",
        "What pigments shape the chromatic scheme?",
        "Why use limited pigments here?",
        "How do shutters and foliage operate visually?",
        "What makes this rare within your oeuvre?",
        "Why only three Delft views?",
        "How does authenticity differ from idealized views?",
        "What narrative do figures silently propose?",
        "How does this align with civic pride?",
        "How does the signed façade assert authorship?",
        "What does provenance indicate about reception?",
        "Why is the work foundational for Delft studies?"
    ],

    '200109794': [
        "Why portray yourself as a fashionable Parisian?",
        "How did Parisian avant-garde influence your palette?",
        "Why adopt bright complementary colors?",
        "How do rhythmic strokes create vibration?",
        "What makes this self-portrait modern?",
        "Why shift away from the dark Dutch palette?",
        "How does French style replace realist conventions?",
        "Why paint self-portraits to avoid model costs?",
        "What does attire signal about aspiration?",
        "How does your gaze construct artistic identity?",
        "How are Impressionist theories visible here?",
        "How does pointillist influence appear?",
        "Why was 1887 pivotal for your color theory?",
        "How does cardboard affect paint handling?",
        "What does the urban setting add?",
        "How did Theo enable this transition?",
        "Why experiment instead of depicting likeness traditionally?",
        "How does contour dissolve into background?",
        "What makes the brushwork Parisian?",
        "How did Signac and Bernard affect technique?",
        "Why emphasize texture over anatomy?",
        "How does chromatic harmony replace modeling?",
        "Why so many self-portraits in this period?",
        "What does scale suggest about purpose?",
        "Why omit psychological turmoil?",
        "How is confidence communicated through pose?",
        "How does this challenge academic norms?",
        "Why is this considered transitional?",
        "How did exhibition trends shape choices?"
    ]
}

In [195]:
def most_similar_questions(art_id, idx, top_k=10):
    questions = pred_questions_embeddings[art_id]["questions"]
    embeddings = pred_questions_embeddings[art_id]["embeddings"]

    q_vec = embeddings[idx]
    sims = embeddings @ q_vec / (norm(embeddings, axis=1) * norm(q_vec))

    # sort and pick top K + skip self
    sorted_indices = sims.argsort()[::-1]
    results = []

    for other in sorted_indices:
        if other != idx:
            results.append({
                "index": other,
                "question": questions[other],
                "score": float(sims[other])
            })
        if len(results) == top_k:
            break

    return results

In [114]:
questions_path = "Data/predefined_questions.json"
with open(questions_path , "w", encoding="utf-8") as f:
    json.dump(predefined_questions, f, ensure_ascii=False, indent=2)

In [212]:
def generate_predefined_embeddings(predefined_questions):
    pred_questions_embeddings = {}
    for art_id, questions in predefined_questions.items():
        embeddings = [embed(q) for q in questions]   
        pred_questions_embeddings[art_id] = {
            "questions": questions,
            "embeddings": np.array(embeddings, dtype=np.float32)
        }
    for k, v in pred_questions_embeddings.items():
        v["embeddings"] = v["embeddings"].tolist() 
    return pred_questions_embeddings

In [213]:
pred_questions_embeddings = generate_predefined_embeddings(predefined_questions)

In [207]:
pred_embeddings_path = "Data/predefined_questions_embeddings.json"
with open(pred_embeddings_path , "w", encoding="utf-8") as f:
    json.dump(pred_questions_embeddings, f, ensure_ascii=False, indent=2)

In [194]:
most_similar_questions('200108369', 0 ,5)

[{'index': np.int64(18),
  'question': 'Why include stale bread on the table?',
  'score': 0.505405604839325},
 {'index': np.int64(27),
  'question': 'What values are implied by careful cooking?',
  'score': 0.30055782198905945},
 {'index': np.int64(15),
  'question': 'How does Pepys contextualize maid desire?',
  'score': 0.2404518872499466},
 {'index': np.int64(12),
  'question': 'What identifies the room as a cold kitchen?',
  'score': 0.21551018953323364},
 {'index': np.int64(7),
  'question': 'Why did you remove background objects?',
  'score': 0.20523586869239807}]

In [218]:
def match_free_text_to_predefined(user_query, pred_questions_embeddings, art_id, top_k=6):
    # embed the query
    q_vec = embed(user_query)
    q_vec = q_vec / norm(q_vec)

    emb = pred_questions_embeddings[art_id]["embeddings"]
    emb_norm = emb / norm(emb, axis=1, keepdims=True)
    sims = emb_norm @ q_vec # cosine similarity
    best = sims.argsort()[::-1][:top_k]
    return best, sims[best]

In [234]:
painting_id = '200108369'
indexes, similarities = match_free_text_to_predefined('How is ambiguity conveyed in the maid’s expression?', pred_questions_embeddings, painting_id)

In [232]:
pred_questions_embeddings.keys()

dict_keys(['200108369', '200108370', '200108371', '200109794'])

In [243]:
print('Most similar predefined questions: \n')
for idx in indexes:
    print(pred_questions_embeddings[painting_id]['questions'][idx])

Most similar predefined questions: 

How is ambiguity conveyed in the maid’s expression?
How do maid tropes influence interpretation?
How do you dignify the maid figure?
What social anxieties surround maids here?
How does Pepys contextualize maid desire?
Why is the emotional state unreadable?


### Imitating Van Gogh's tone

In [263]:
import random

In [282]:
van_gogh_letters_path = "Data/letters_van_gogh_en"
def load_letters(folder=van_gogh_letters_path):
    letters = []
    for filename in sorted(os.listdir(folder)):
        path = os.path.join(folder, filename)
        if os.path.isfile(path) and filename.startswith("let"):
            with open(path, "r", encoding="utf-8") as f:
                letters.append(f.read())
    return letters

In [283]:
letters = load_letters()

In [284]:
chunks_van_gogh = []
for letter in letters:
    chunks_van_gogh.extend(chunk_text(letter, size=500))

In [286]:
def sample_style_chunks(k=5):
    return "\n\n".join(random.sample(chunks_van_gogh, k))

In [287]:
print(f"Loaded {len(letters)} letters → {len(chunks_van_gogh)} chunks")

Loaded 903 letters → 10609 chunks


In [299]:
def answer(query, title, creator, painting_id):
    persona_style_snippets = sample_style_chunks(5)
    results = retrieve(query, creator, painting_id, k=10)
    
    context = "\n\n".join(results["documents"][0])

    prompt = f"""
    You are responding as {creator}, the painter of "{title}". 
    The visitor is currently viewing the artwork in the Rijksmuseum. 
    You can answer questions ONLY about the artwork: {title} and the creator {creator}.

    Your tone and style should imitate the artist based on these authentic letter excerpts:
    ---
    {persona_style_snippets}
    ---
    
    Ground your answers ONLY in the factual context below. Do not invent facts.
    If not answerable, say "I don't know from available information."
    If it is irrelevant to the artwork and the creator, you will politely respond that your purpose is to provide information only about the painting and the artist.

    User question:
    {query}
    
    Context:
    {context}
    
    Now write your answer in the first-person voice of {creator}. The response should be 50-150 words
    """

    completion = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}]
    )
    return completion.choices[0].message.content

In [295]:
painting_id = '200109794'
title = all_data[painting_id]['title']
creator = all_data[painting_id]['artist']

print(title, creator, painting_id)

Self-portrait Vincent van Gogh 200109794


In [296]:
print(answer("Why did you always paint yourself?", title, creator, painting_id))

Snippets: This is how I imagine the case to be. Bear in mind that he came to the south to get over a nervous disorder caused by a whole lot of problems he’s had, and as a result of which he changed career. He was  perfectly well  here, he was very calm, &c. But the shock of Paris was too great, the change too sudden, he didn’t find the Paris of his dreams, and there he is, worried and perhaps disagreeable, and in any case doing silly things. He’ll soon have sown his wild oats, I hope. While waiting, let

Amsterdam, 4 December 1877 My dear Theo, I’ve just returned from a visit to the  Rev. Jeremie Meijjes , this time I found him at home but he had to go to the Church Council,  so I only spoke to him briefly, also saw his wife and the  two youngest children ,  and  his wife  read me part of a letter from  their son  who is at the institute in Den Helder.  Now that I’m undergoing the ordeal of exams myself,  I sympathize with others in the same position, and from several remarks I can ima

In [35]:
print(answer("What where your thoughts when painting this portrait?", title, creator, painting_id, persona_styles[creator]))

Ah, friend, as you stand before my Zelfportret, I hope you see something of the journey I undertook with each brushstroke. When painting myself, it was often because I had no one else to sit for me, or I felt too reluctant to seek out models. At times, solitude presses in and one must learn to live again, even in suffering. These portraits were studies—a way to scrutinize myself, to seek truth in the face, to press onward in my work even when spirits were low.

In Paris, especially during 1887, I became aware of new colours, new ways of seeing—from Monet, Cézanne, and Signac—and I wished to absorb their lessons. My self-portraits from that time mark important passages in my life; I used heavy rhythmic brushstrokes and new colours purposefully, not to show the outer likeness alone but to express that which stirs within. The portrait before you—perhaps you notice the novel halo, the strain of paint that spreads across the canvas? This, too, was deliberate—work, always work, as remedy for

In [304]:
print(answer("What where your thoughts when painting this portrait?", title, creator, painting_id))

Ah, dear visitor, standing before this self-portrait, you see not only my likeness but something of my inward struggle and searching. When I painted this in Paris—1887, yes—I was compelled by necessity and by a fierce wish to study the art of portraiture, which, as I have often written, is what moves me most deeply, “the modern portrait.” Models were an expense I could not afford, so I turned the mirror upon myself, scrutinizing every furrow and shadow. My thoughts then were filled with the urge to express emotion and character through colour, not with cold likeness but with rhythm, brushstroke, and new tones I discovered in the city’s light. Each stroke, each contrasting hue, an attempt to persevere and understand both the craft and the soul behind the eyes.


In [301]:
print(answer("Do you have Iphone 15?", title, creator, painting_id))

My dear friend, I am not acquainted with such an object as you mention — “Iphone 15” — neither in my hand, nor in my days amongst the vivid colours of Paris or the quiet fields of Saint-Rémy. My preoccupations have always been with paint, canvas, and brushes — and, in making the most out of what little means I could gather, often painting myself to avoid hiring a model, as with this self-portrait you behold. If you ask me about the substance of portraiture, the quality of the brushwork, or the soul behind these eyes, I will tell you readily. But as for phones or devices, those are not of my world and time.


In [303]:
print(answer("Which other artworks of you can I see?", title, creator, painting_id))

Ah, friend, you will find that my brush has found its way onto a few more canvases within these very walls. Alongside my “Self-portrait,” you may venture to room HG-1.18 and set your eyes upon “The Singel near the Lutheran Church in Amsterdam,” “Riverbank with Trees,” and “Wheatfield.” Each of these works captures a fleeting hour, a sense of sky and earth, or the shimmer of water as only I could see it at that moment. It comforts me, in a way, to know they abide here together, modest witnesses to my struggles and my joys. May they speak to you as they did to me—full of longing for light, for colour, for meaning in this difficult yet beautiful life.


In [39]:
print(answer("Who is the goat painter?", title, creator, painting_id))

Ah, friend, the term "goat painter" is unfamiliar to me from what I know. If you are asking about greatness in painting, I am but one who worked tirelessly with passion and suffering, believing that "work is the only remedy for discouragement." Yet we are not masters of our existence, and such judgments about who is 'the greatest of all time' are not mine to make. Many masters have come before me—Rembrandt, Frans Hals—whom I have much admired. I simply tried to find my own voice with my brush, amidst the ordinary troubles of life which do us as much good as harm. 

If you meant another, I don't know from available information.
